In [4]:
!pip install transformers pymilvus sentence-transformers huggingface-hub langchain_community langchain-text-splitters pypdf


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 16.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 15.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 11

In [60]:
! pip install bs4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


1. Extracting text

In [158]:
import glob
from langchain_text_splitters import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup



In [63]:
path = '/Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/'
files = glob.glob(path +'*.html')

In [202]:
text_lines = []

for file in files:
    with open(file, 'r', encoding='ISO-8859-1') as f:
        print(f"Processing: {file}")
        raw_html = f.read()

        # Clean HTML to plain text
        soup = BeautifulSoup(raw_html, "html.parser")
        clean_text = soup.get_text(separator=" ", strip=True)

        # Optional: remove excess whitespace
        clean_text = " ".join(clean_text.split())

        # Split cleaned text
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=20000, chunk_overlap=50)
        chunks = text_splitter.split_text(clean_text)

        text_lines.extend(chunks)

        
print(text_lines[:5])  # Print first 5 chunks for verification

        
        

Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Résumé des caractéristiques du produit - KARDEGIC 160 mg, poudre pour solution buvable en sachet - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - SERESTA 10 mg, comprimé - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - KARDEGIC 160 mg, poudre pour solution buvable en sachet - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Fiche info - DOLIPRANE 500 mg, comprimé - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeplearningproject/data/Résumé des caractéristiques du produit - DAFALGAN 500 mg, comprimé - Base de données publique des médicaments.html
Processing: /Users/naomi/Desktop/EM/Deep learning/deeple

2. Embedding a sentence

In [187]:
from sentence_transformers import SentenceTransformer

In [203]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 
s = text_lines[0]
e = embedding_model.encode([s])


Embedding for first chunk: [[-5.64096728e-03 -5.37636578e-02 -3.59106101e-02 -6.33561388e-02
  -6.85769618e-02  2.65338700e-02  2.50280909e-02  1.67792171e-01
   4.04125340e-02 -2.17751637e-02  3.17669958e-02 -9.73141007e-03
   4.97239605e-02  3.42365075e-03 -5.91514409e-02 -1.56180356e-02
   4.91707250e-02 -1.04245031e-02  2.26925686e-02  4.52948399e-02
   3.71564068e-02  6.50669038e-02 -2.88937651e-02  4.87636961e-02
  -2.92462744e-02  1.01817204e-02  5.29631600e-03 -5.91232404e-02
   3.49166468e-02 -1.95771102e-02  6.55516684e-02  5.14251105e-02
   5.42769879e-02 -8.20764452e-02  5.46876751e-02  1.22944042e-01
  -1.63360253e-01 -5.73268998e-03 -6.01159818e-02  5.27237616e-02
  -9.47566144e-03 -4.92370799e-02 -5.86613342e-02 -5.09079359e-02
   6.56111017e-02 -1.10057771e-01 -4.71003912e-02 -3.43009233e-02
  -5.52962869e-02  9.18113142e-02 -8.92752931e-02 -4.62643569e-03
  -5.10504358e-02  3.11939456e-02  7.87817966e-03 -8.50793645e-02
  -1.40313864e-01 -1.28896246e-02 -6.03171848e-02

3. Creating a Milvus data collection


In [189]:
from pymilvus import MilvusClient

In [190]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

def emb_text (text_lines):
    embeddings = embedding_model.encode(text_lines, batch_size=32, show_progress_bar=False)
    return embeddings

In [191]:
milvus_client = MilvusClient(uri="./my_milvus_db.db")
collection_name = "rag_collection"
milvus_client.create_collection(
 collection_name=collection_name,
 dimension= 384, # The size of the embedding
 metric_type="IP", # Inner product distance
 consistency_level="Strong", # Strong consistency level
)


In [192]:
data = []
# In the following example, emb_text is a function that needs to be written, based on
# an embedding model
for i, line in enumerate(text_lines):
    data.append({"id": i, "vector": emb_text(line), "text": line})


In [193]:
insert_res = milvus_client.insert(collection_name=collection_name, data=data)

4. Retrieving data for a query

In [206]:
question = "Que faire en cas de fievre"
search_res = milvus_client.search(
 collection_name=collection_name,
 data=[
 emb_text(question)
 ],
 limit=5, # Return top 3 results
 search_params={"metric_type": "IP", "params": {}}, # Inner product distance
 output_fields=["text"], # Return the text field
)

print(search_res)

data: ['[{\'id\': 1616, \'distance\': 0.5525888204574585, \'entity\': {\'text\': \'Européen Sans objet. La dernière date à laquelle cette notice a été révisée est : [à compléter ultérieurement par le titulaire] {mois AAAA}. Autres Des informations détaillées sur ce médicament sont disponibles sur le site Internet de l\\x92ANSM (France). CONSEIL D\\x92EDUCATION SANITAIRE : "QUE FAIRE EN CAS DE FIEVRE" : La température normale du corps est variable d\\\'un individu à l\\\'autre et comprise entre 36,5°C et 37,5°C. Une élévation de plus de 0,8°C est considérée comme une fièvre. Chez l\\\'adulte et chez l\\\'enfant de plus de 20 kg (environ 6 ans): si les troubles qu\\\'elle entraîne sont trop gênants, vous pouvez prendre un médicament qui contient de l\\\'ibuprofène en respectant les posologies indiquées. Avec ce médicament, la fièvre doit baisser rapidement. Néanmoins : · si d\\\'autres signes apparaissent (comme une éruption cutanée), · si la fièvre persiste plus de 3 jours ou si elle s\

In [222]:
import re

In [224]:
context = [result['entity']['text'] for result in search_res[0]]
print(context)
context_str = " ".join(context)  # turn list into a single string
context = re.sub(r"{.*?}", "", context_str)



['Européen Sans objet. La dernière date à laquelle cette notice a été révisée est : [à compléter ultérieurement par le titulaire] {mois AAAA}. Autres Des informations détaillées sur ce médicament sont disponibles sur le site Internet de l\x92ANSM (France). CONSEIL D\x92EDUCATION SANITAIRE : "QUE FAIRE EN CAS DE FIEVRE" : La température normale du corps est variable d\'un individu à l\'autre et comprise entre 36,5°C et 37,5°C. Une élévation de plus de 0,8°C est considérée comme une fièvre. Chez l\'adulte et chez l\'enfant de plus de 20 kg (environ 6 ans): si les troubles qu\'elle entraîne sont trop gênants, vous pouvez prendre un médicament qui contient de l\'ibuprofène en respectant les posologies indiquées. Avec ce médicament, la fièvre doit baisser rapidement. Néanmoins : · si d\'autres signes apparaissent (comme une éruption cutanée), · si la fièvre persiste plus de 3 jours ou si elle s\'aggrave, · si les maux de tête deviennent violents, ou en cas de vomissements, CONSULTEZ IMMEDIA

5. Create a prompt 

In [225]:
from huggingface_hub import notebook_login
notebook_login()

from transformers import AutoTokenizer, AutoModelForCausalLM

In [234]:
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [227]:
PROMPT = """
Utilise les informations dans <context> pour répondre à la question dans <question>.
<context>
{context}
</context>
<question>
{question}
</question>" \
"""

In [228]:
PROMPT = PROMPT.format(question=question, context=context)
print(PROMPT)


Utilise les informations dans <context> pour répondre à la question dans <question>.
<context>
Européen Sans objet. La dernière date à laquelle cette notice a été révisée est : [à compléter ultérieurement par le titulaire] . Autres Des informations détaillées sur ce médicament sont disponibles sur le site Internet de lANSM (France). CONSEIL DEDUCATION SANITAIRE : "QUE FAIRE EN CAS DE FIEVRE" : La température normale du corps est variable d'un individu à l'autre et comprise entre 36,5°C et 37,5°C. Une élévation de plus de 0,8°C est considérée comme une fièvre. Chez l'adulte et chez l'enfant de plus de 20 kg (environ 6 ans): si les troubles qu'elle entraîne sont trop gênants, vous pouvez prendre un médicament qui contient de l'ibuprofène en respectant les posologies indiquées. Avec ce médicament, la fièvre doit baisser rapidement. Néanmoins : · si d'autres signes apparaissent (comme une éruption cutanée), · si la fièvre persiste plus de 3 jours ou si elle s'aggrave, · si les maux de t

In [235]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "gpt2" 
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map=None).to(device)



In [236]:
inputs = tokenizer(PROMPT.format(question=question, context=context), return_tensors="pt").to(device)
output_ids = model.generate(**inputs, max_new_tokens=100)


response = tokenizer.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


était en ce n'est pas de fievre,
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context>
<context><context><context><
